In [208]:
import numpy as np
import random
import cv2
from compute_avg_reproj_error import *

In [209]:
def imgShow(img,title=""):
    img=((img-img.min())/(img.max()-img.min())* 255.).astype(np.uint8) 
    cv2.imshow(title,img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [210]:
def getAMatrix(srcP,destP):
    A=[]
    for p,pDot in zip(srcP,destP):
        dx,dy=pDot; x,y=p
        Ai=np.array([[x*dx,y*dx,dx,x*dy,y*dy,dy,x,y,1]])
        if len(A)==0:  A=Ai
        else:  A=np.vstack([A,Ai])
    return A

In [211]:
def getNormalizingMatrix(matrix):
    matrix=np.append(matrix,np.ones((len(matrix),1)),axis=1)
    mean=np.mean(matrix)
    subtractingMatrix=np.array([[1,0,-mean],[0,1,-mean],[0,0,1]])
    subtractedMatrix=subtractingMatrix@matrix.T
    maxi,mini=np.max(subtractedMatrix),np.min(subtractedMatrix)
    scalingValue=1/(maxi-mini)
    scalingMatrix1=np.array([[1,0,-mini],[0,1,-mini],[0,0,1]])
    scalingMatrix2=np.array([[scalingValue,0,0],[0,scalingValue,0],[0,0,1]]) #x,y 각각 1 => 최대 sqrt(2)

    return scalingMatrix2@scalingMatrix1@subtractingMatrix

In [212]:
def compute_F_raw(M):
    
    srcP,destP=M[:,:2],M[:,2:]
    A=getAMatrix(srcP,destP)
    
    U,s,V=np.linalg.svd(A)
    F=np.reshape(V[-1],(3,3))
    
    U,s,V=np.linalg.svd(F)
    s=np.diag([*s[:2],0])
    Fhat=U@s@V
    
    return Fhat/Fhat[-1,-1]

In [213]:
def getNormalizedMatrix(P,matrix):
    P=np.append(P,np.ones((len(P),1)),axis=1)
    normalized=matrix@P.T
    if 0 not in normalized[-1]:
        normalized/=normalized[-1]
    return normalized[:2].T

In [214]:
def compute_F_norm(M):
    srcP,destP=M[:,:2],M[:,2:]
    
    srcPMatrix=getNormalizingMatrix(srcP)
    normalizedSrcP=getNormalizedMatrix(srcP,srcPMatrix)
    destPMatrix=getNormalizingMatrix(destP)
    normalizedDestP=getNormalizedMatrix(destP,destPMatrix)
    
    M=np.hstack([normalizedSrcP,normalizedDestP])
    F=compute_F_raw(M)
    return destPMatrix.T@F@srcPMatrix

In [215]:
def compute_F_mine(M):
    th=1000
    F=None
    for i in range(5000):
        randomM=M[np.random.choice(len(M),8,replace=False)]
        f=compute_F_norm(randomM)
        err=compute_avg_reproj_error(randomM,f)
        if err<th:
            F=f
            th=err
    return F


In [216]:
def computeEpilpolarLine(F,p):
    x,y,x1,y1=p
    return np.array([x,y,1])@F,F@np.array([x1,y1,1])

In [217]:
def drawEpilpolarLine(img,line,p,color):
    _,w,_=img.shape
    a,b,c=line
    p1=(0,int(-c/b))
    p2=(w,int(-(a*w+c)/b))
    img=cv2.line(img,p1,p2,color,1)
    img=cv2.circle(img,tuple(map(int,[*p])),5,color,-1)
    return img

In [218]:
def getResultImgs(img1,img2,M,F):
    colorList=[(255,0,0),(0,255,0),(0,0,255)]
    for p,color in zip(M[np.random.choice(len(M),3,replace=False)],colorList):
        l1,m1=computeEpilpolarLine(F,p)
        img1=drawEpilpolarLine(img1,l1,p[:2],color)
        img2=drawEpilpolarLine(img2,m1,p[2:],color)
    return img1,img2

In [219]:
img1=cv2.imread('temple1.png')
img2=cv2.imread('temple2.png')
M=np.array(np.loadtxt('temple_matches.txt'))
rawF=compute_F_raw(M)
normF=compute_F_norm(M)

In [220]:
img1=cv2.imread('temple1.png')
img2=cv2.imread('temple2.png')
M=np.array(np.loadtxt('temple_matches.txt'))
rawF=compute_F_raw(M)
result1,result2=getResultImgs(img1,img2,M,rawF)
imgShow(result1)
imgShow(result2)

In [221]:
img1=cv2.imread('temple1.png')
img2=cv2.imread('temple2.png')
M=np.array(np.loadtxt('temple_matches.txt'))
normF=compute_F_norm(M)
result1,result2=getResultImgs(img1,img2,M,normF)
imgShow(result1)
imgShow(result2)

In [222]:
img1=cv2.imread('temple1.png')
img2=cv2.imread('temple2.png')
M=np.array(np.loadtxt('temple_matches.txt'))
mineF=compute_F_mine(M)
result1,result2=getResultImgs(img1,img2,M,mineF)
imgShow(result1)
imgShow(result2)

In [223]:
print(f"raw: {compute_avg_reproj_error(M,rawF)}")
print(f"norm: {compute_avg_reproj_error(M,normF)}")
print(f"mine: {compute_avg_reproj_error(M,mineF)}")

raw: 12.33253808689864
norm: 0.5163948546414058
mine: 0.5163948546414058
